In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Vitamin_D_Levels"
cohort = "GSE123993"

# Input paths
in_trait_dir = "../../input/GEO/Vitamin_D_Levels"
in_cohort_dir = "../../input/GEO/Vitamin_D_Levels/GSE123993"

# Output paths
out_data_file = "../../output/preprocess/Vitamin_D_Levels/GSE123993.csv"
out_gene_data_file = "../../output/preprocess/Vitamin_D_Levels/gene_data/GSE123993.csv"
out_clinical_data_file = "../../output/preprocess/Vitamin_D_Levels/clinical_data/GSE123993.csv"
json_path = "../../output/preprocess/Vitamin_D_Levels/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Let's first list the directory contents to understand what files are available
import os

print("Files in the cohort directory:")
files = os.listdir(in_cohort_dir)
print(files)

# Adapt file identification to handle different naming patterns
soft_files = [f for f in files if 'soft' in f.lower() or '.soft' in f.lower() or '_soft' in f.lower()]
matrix_files = [f for f in files if 'matrix' in f.lower() or '.matrix' in f.lower() or '_matrix' in f.lower()]

# If no files with these patterns are found, look for alternative file types
if not soft_files:
    soft_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]
if not matrix_files:
    matrix_files = [f for f in files if f.endswith('.txt') or f.endswith('.gz')]

print("Identified SOFT files:", soft_files)
print("Identified matrix files:", matrix_files)

# Use the first files found, if any
if len(soft_files) > 0 and len(matrix_files) > 0:
    soft_file = os.path.join(in_cohort_dir, soft_files[0])
    matrix_file = os.path.join(in_cohort_dir, matrix_files[0])
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # 3. Obtain the sample characteristics dictionary from the clinical dataframe
    sample_characteristics_dict = get_unique_values_by_row(clinical_data)
    
    # 4. Explicitly print out all the background information and the sample characteristics dictionary
    print("\nBackground Information:")
    print(background_info)
    print("\nSample Characteristics Dictionary:")
    print(sample_characteristics_dict)
else:
    print("No appropriate files found in the directory.")


Files in the cohort directory:
['GSE123993_family.soft.gz', 'GSE123993_series_matrix.txt.gz']
Identified SOFT files: ['GSE123993_family.soft.gz']
Identified matrix files: ['GSE123993_series_matrix.txt.gz']

Background Information:
!Series_title	"No effect of calcifediol supplementation on skeletal muscle transcriptome in vitamin D deficient frail older adults."
!Series_summary	"Vitamin D deficiency is common among older adults and has been linked to muscle weakness. Vitamin D supplementation has been proposed as a strategy to improve muscle function in older adults. The aim of this study was to investigate the effect of calcifediol (25-hydroxycholecalciferol) on whole genome gene expression in skeletal muscle of vitamin D deficient frail older adults. A double-blind placebo controlled trial was conducted in vitamin D deficient frail older adults (aged above 65), characterized by blood 25-hydroxycholecalciferol concentrations between 20 and 50 nmol/L. Subjects were randomized across the

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Check gene expression data availability
# Based on the background information, this is a microarray study using Affymetrix HuGene 2.1ST arrays
is_gene_available = True  # Microarray gene expression data appears to be available

# 2. Analyze variable availability and define conversion functions

# 2.1 Identify trait data (Vitamin D levels)
# From the background info, this is about vitamin D supplementation and calcifediol (25-hydroxycholecalciferol)
# Looking at the sample characteristics, intervention group (index 3) and time of sampling (index 4) 
# can be used to track vitamin D status/intervention
trait_row = 3  # intervention group

# Define conversion function for trait (0 for placebo, 1 for supplementation)
def convert_trait(val):
    if isinstance(val, str):
        if ':' in val:
            val = val.split(':', 1)[1].strip()
        if '25-hydroxycholecalciferol' in val or '25(OH)D3' in val:
            return 1  # Vitamin D supplementation
        elif 'Placebo' in val:
            return 0  # Placebo control
    return None  # Unknown or invalid value

# 2.2 Identify age data
# Age is not explicitly provided in the sample characteristics
age_row = None  # Age data not available

def convert_age(val):
    # Function defined but won't be used since age data is not available
    return None

# 2.3 Identify gender data
# Gender is available in index 1 (Sex)
gender_row = 1  # Sex data

def convert_gender(val):
    if isinstance(val, str):
        if ':' in val:
            val = val.split(':', 1)[1].strip()
        if val.lower() == 'female':
            return 0
        elif val.lower() == 'male':
            return 1
    return None  # Unknown or invalid value

# 3. Save metadata on dataset usability
# Check if trait data is available
is_trait_available = trait_row is not None

# Save metadata with initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if trait data is available
if trait_row is not None:
    # Load the clinical data (assuming clinical_data is available from previous step)
    # Note: clinical_data is assumed to be available from a previous step
    
    # Extract clinical features
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical features to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM3518336': [1.0, 1.0], 'GSM3518337': [1.0, 1.0], 'GSM3518338': [1.0, 0.0], 'GSM3518339': [1.0, 0.0], 'GSM3518340': [1.0, 0.0], 'GSM3518341': [1.0, 0.0], 'GSM3518342': [1.0, 1.0], 'GSM3518343': [1.0, 1.0], 'GSM3518344': [0.0, 1.0], 'GSM3518345': [0.0, 1.0], 'GSM3518346': [0.0, 1.0], 'GSM3518347': [0.0, 1.0], 'GSM3518348': [0.0, 0.0], 'GSM3518349': [0.0, 0.0], 'GSM3518350': [1.0, 1.0], 'GSM3518351': [1.0, 1.0], 'GSM3518352': [0.0, 0.0], 'GSM3518353': [0.0, 0.0], 'GSM3518354': [1.0, 1.0], 'GSM3518355': [1.0, 1.0], 'GSM3518356': [1.0, 0.0], 'GSM3518357': [1.0, 0.0], 'GSM3518358': [0.0, 0.0], 'GSM3518359': [0.0, 0.0], 'GSM3518360': [0.0, 1.0], 'GSM3518361': [0.0, 1.0], 'GSM3518362': [0.0, 0.0], 'GSM3518363': [0.0, 0.0], 'GSM3518364': [0.0, 1.0], 'GSM3518365': [0.0, 1.0], 'GSM3518366': [0.0, 1.0], 'GSM3518367': [0.0, 1.0], 'GSM3518368': [0.0, 0.0], 'GSM3518369': [0.0, 0.0], 'GSM3518370': [1.0, 1.0], 'GSM3518371': [1.0, 1.0], 'GSM3518372': [1.0, 1.0

### Step 3: Gene Data Extraction

In [4]:
# Use the helper function to get the proper file paths
soft_file_path, matrix_file_path = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
try:
    gene_data = get_genetic_data(matrix_file_path)
    
    # Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
    
    # Print shape to understand the dataset dimensions
    print(f"\nGene expression data shape: {gene_data.shape}")
    
except Exception as e:
    print(f"Error extracting gene data: {e}")


First 20 gene/probe identifiers:
Index(['16650001', '16650003', '16650005', '16650007', '16650009', '16650011',
       '16650013', '16650015', '16650017', '16650019', '16650021', '16650023',
       '16650025', '16650027', '16650029', '16650031', '16650033', '16650035',
       '16650037', '16650041'],
      dtype='object', name='ID')

Gene expression data shape: (53617, 44)


### Step 4: Gene Identifier Review

In [5]:
# Examining the gene identifiers
# The identifiers '16650001', '16650003', etc. appear to be numeric probe IDs
# These are not standard human gene symbols (which would be alphanumeric like BRCA1, TP53, etc.)
# These appear to be Affymetrix or similar microarray probe IDs that need mapping to gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. This part examines the data more thoroughly to determine what type of data it contains
try:
    # First, let's check a few rows of the gene_data we extracted in Step 3
    print("Sample of gene expression data (first 5 rows, first 5 columns):")
    print(gene_data.iloc[:5, :5])
    
    # Analyze the SOFT file to identify the data type and mapping information
    platform_info = []
    with gzip.open(soft_file_path, 'rt', encoding='latin-1') as f:
        for line in f:
            if line.startswith("!Platform_title") or line.startswith("!Series_title") or "description" in line.lower():
                platform_info.append(line.strip())
    
    print("\nPlatform information:")
    for line in platform_info:
        print(line)
    
    # Extract the gene annotation using the library function
    gene_annotation = get_gene_annotation(soft_file_path)
    
    # Display column names of the annotation dataframe
    print("\nGene annotation columns:")
    print(gene_annotation.columns.tolist())
    
    # Preview the annotation dataframe
    print("\nGene annotation preview:")
    annotation_preview = preview_df(gene_annotation)
    print(annotation_preview)
    
    # Check if ID column exists in the gene_annotation dataframe
    if 'ID' in gene_annotation.columns:
        # Check if any of the IDs in gene_annotation match those in gene_data
        sample_ids = list(gene_data.index[:10])
        matching_rows = gene_annotation[gene_annotation['ID'].isin(sample_ids)]
        print(f"\nMatching rows in annotation for sample IDs: {len(matching_rows)}")
        
        # Look for gene symbol column
        gene_symbol_candidates = [col for col in gene_annotation.columns if 'gene' in col.lower() or 'symbol' in col.lower() or 'name' in col.lower()]
        print(f"\nPotential gene symbol columns: {gene_symbol_candidates}")
    
except Exception as e:
    print(f"Error analyzing gene annotation data: {e}")
    gene_annotation = pd.DataFrame()

# Based on our analysis, determine if this is really gene expression data
# Check the platform description and match with the data we've extracted
is_gene_expression = False
for info in platform_info:
    if 'expression' in info.lower() or 'transcript' in info.lower() or 'mrna' in info.lower():
        is_gene_expression = True
        break

print(f"\nIs this dataset likely to contain gene expression data? {is_gene_expression}")

# If this isn't gene expression data, we need to update our metadata
if not is_gene_expression:
    print("\nNOTE: Based on our analysis, this dataset doesn't appear to contain gene expression data.")
    print("It appears to be a different type of data (possibly SNP array or other genomic data).")
    # Update is_gene_available for metadata
    is_gene_available = False
    
    # Save the updated metadata
    validate_and_save_cohort_info(
        is_final=False,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available
    )


Sample of gene expression data (first 5 rows, first 5 columns):
          GSM3518336  GSM3518337  GSM3518338  GSM3518339  GSM3518340
ID                                                                  
16650001    1.512274    0.253623    2.664750    1.312637    0.675071
16650003    1.003784    1.028232    1.341918    1.114636    1.802068
16650005    0.604331    1.397437    1.651186    3.274191    0.394109
16650007    1.058137    0.588526    1.149379    0.761508    2.417583
16650009    0.469632    0.698155    0.888779    1.154360    0.859562



Platform information:
!Series_title = No effect of calcifediol supplementation on skeletal muscle transcriptome in vitamin D deficient frail older adults.
!Platform_title = [HuGene-2_1-st] Affymetrix Human Gene 2.1 ST Array [transcript (gene) version]
!Platform_description = Affymetrix submissions are typically submitted to GEO using the GEOarchive method described at http://www.ncbi.nlm.nih.gov/projects/geo/info/geo_affy.html
!Platform_description =
!Platform_description = September 06, 2013: HuGene-2_1-st-v1.na33.2.hg19.transcript.csv



Gene annotation columns:
['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'gene_assignment', 'mrna_assignment', 'swissprot', 'unigene', 'GO_biological_process', 'GO_cellular_component', 'GO_molecular_function', 'pathway', 'protein_domains', 'crosshyb_type', 'category', 'GB_ACC', 'SPOT_ID']

Gene annotation preview:
{'ID': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'probeset_id': ['16657436', '16657440', '16657445', '16657447', '16657450'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['12190', '29554', '69091', '160446', '317811'], 'stop': ['13639', '31109', '70008', '161525', '328581'], 'total_probes': [25.0, 28.0, 8.0, 13.0, 36.0], 'gene_assignment': ['NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 1 // 1p36.33 // 100287102 /// NR_034090 // DDX11L9 // DEAD/H (Asp-Glu-Ala-Asp/His) box helicase 11 like 9 // 15q26.3 // 100288486 /// NR_051985 // DDX11L9 // DEAD

### Step 6: Gene Identifier Mapping

In [7]:
# Step 1: Identify which columns in the gene annotation data correspond to probe IDs and gene symbols
# Based on the data preview:
# - The 'ID' column in gene_annotation contains probe IDs that match gene_data.index
# - The 'gene_assignment' column contains information about genes including gene symbols

# Extract the mapping between probe IDs and gene symbols
# We'll use the 'ID' column for probe IDs and 'gene_assignment' for gene symbols
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# Print the first few rows of the mapping dataframe to verify its structure
print("Gene mapping dataframe preview (first 5 rows):")
print(mapping_df.head())
print(f"Total number of probe-gene mappings: {len(mapping_df)}")

# Step 2 & 3: Convert probe-level measurements to gene-level expression data
# Apply the mapping to convert probe IDs to gene symbols and aggregate expression values
try:
    print("Converting probe-level data to gene-level data...")
    gene_data = apply_gene_mapping(gene_data, mapping_df)
    
    # Print information about the resulting gene expression dataframe
    print(f"Gene expression data shape after mapping: {gene_data.shape}")
    print("First 10 gene symbols in the mapped data:")
    print(gene_data.index[:10])
    
    # Save the gene expression data to file
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")
    
except Exception as e:
    print(f"Error during gene mapping: {e}")


Gene mapping dataframe preview (first 5 rows):
         ID                                               Gene
0  16657436  NR_046018 // DDX11L1 // DEAD/H (Asp-Glu-Ala-As...
1  16657440  ENST00000473358 // MIR1302-11 // microRNA 1302...
2  16657445  NM_001005484 // OR4F5 // olfactory receptor, f...
3  16657447                                                ---
4  16657450  AK302511 // LOC100132062 // uncharacterized LO...
Total number of probe-gene mappings: 53617
Converting probe-level data to gene-level data...


Gene expression data shape after mapping: (81076, 44)
First 10 gene symbols in the mapped data:
Index(['A-', 'A-2', 'A-52', 'A-E', 'A-I', 'A-II', 'A-IV', 'A-V', 'A0', 'A1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Vitamin_D_Levels/gene_data/GSE123993.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
try:
    # Now let's normalize the gene data using the provided function
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    print(f"First few gene symbols after normalization: {list(normalized_gene_data.index[:10])}")
    
    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error in gene normalization: {e}")
    # If normalization fails, use the original gene data
    normalized_gene_data = gene_data
    print("Using original gene data without normalization")

# 2. Load the clinical data - make sure we have the correct format
try:
    # Load the clinical data we saved earlier to ensure correct format
    clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
    print("Loaded clinical data:")
    print(clinical_data.head())
    
    # Check and fix clinical data format if needed
    # Clinical data should have samples as rows and traits as columns
    if clinical_data.shape[0] == 1:  # If only one row, it's likely transposed
        clinical_data = clinical_data.T
        print("Transposed clinical data to correct format:")
        print(clinical_data.head())
except Exception as e:
    print(f"Error loading clinical data: {e}")
    # If loading fails, recreate the clinical features
    clinical_data = geo_select_clinical_features(
        clinical_df, 
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    ).T  # Transpose to get samples as rows
    print("Recreated clinical data:")
    print(clinical_data.head())

# Ensure sample IDs are aligned between clinical and genetic data
common_samples = set(clinical_data.index).intersection(normalized_gene_data.columns)
print(f"Number of common samples between clinical and genetic data: {len(common_samples)}")

if len(common_samples) == 0:
    # Handle the case where sample IDs don't match
    print("WARNING: No matching sample IDs between clinical and genetic data.")
    print("Clinical data index:", clinical_data.index.tolist())
    print("Gene data columns:", list(normalized_gene_data.columns[:5]) + ["..."])
    
    # Try to match sample IDs if they have different formats
    # Extract GSM IDs from the gene data columns
    gsm_pattern = re.compile(r'GSM\d+')
    gene_samples = []
    for col in normalized_gene_data.columns:
        match = gsm_pattern.search(str(col))
        if match:
            gene_samples.append(match.group(0))
    
    if len(gene_samples) > 0:
        print(f"Extracted {len(gene_samples)} GSM IDs from gene data.")
        normalized_gene_data.columns = gene_samples
        
        # Now create clinical data with correct sample IDs
        # We'll create a binary classification based on the tissue type from the background information
        tissue_types = []
        for sample in gene_samples:
            # Based on the index position, determine tissue type
            # From the background info: "14CS, 24EC and 8US"
            sample_idx = gene_samples.index(sample)
            if sample_idx < 14:
                tissue_types.append(1)  # Carcinosarcoma (CS)
            else:
                tissue_types.append(0)  # Either EC or US
        
        clinical_data = pd.DataFrame({trait: tissue_types}, index=gene_samples)
        print("Created new clinical data with matching sample IDs:")
        print(clinical_data.head())

# 3. Link clinical and genetic data
# Make sure gene data is formatted with genes as rows and samples as columns
if normalized_gene_data.index.name != 'Gene':
    normalized_gene_data.index.name = 'Gene'

# Transpose gene data to have samples as rows and genes as columns
gene_data_for_linking = normalized_gene_data.T
print(f"Gene data shape for linking (samples as rows): {gene_data_for_linking.shape}")

# Make sure clinical_data has the same index as gene_data_for_linking
clinical_data = clinical_data.loc[clinical_data.index.isin(gene_data_for_linking.index)]
gene_data_for_linking = gene_data_for_linking.loc[gene_data_for_linking.index.isin(clinical_data.index)]

# Now link by concatenating horizontally
linked_data = pd.concat([clinical_data, gene_data_for_linking], axis=1)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 columns):")
sample_cols = [trait] + list(linked_data.columns[1:5]) if len(linked_data.columns) > 5 else list(linked_data.columns)
print(linked_data[sample_cols].head())

# 4. Handle missing values
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# Check if we still have data
if linked_data.shape[0] == 0 or linked_data.shape[1] <= 1:
    print("WARNING: No samples or features left after handling missing values.")
    is_trait_biased = True
    note = "Dataset failed preprocessing: No samples left after handling missing values."
else:
    # 5. Determine whether the trait and demographic features are biased
    is_trait_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
    print(f"Is trait biased: {is_trait_biased}")
    note = "This dataset contains gene expression data from uterine corpus tissues, comparing carcinosarcoma with endometrioid adenocarcinoma and sarcoma."

# 6. Conduct quality check and save the cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True,  
    is_trait_available=True,
    is_biased=is_trait_biased, 
    df=linked_data,
    note=note
)

# 7. Save the linked data if it's usable
print(f"Data quality check result: {'Usable' if is_usable else 'Not usable'}")
if is_usable:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Data not saved due to quality issues.")

Gene data shape after normalization: (23274, 44)
First few gene symbols after normalization: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2ML1-AS1', 'A2ML1-AS2', 'A2MP1', 'A4GALT']


Normalized gene data saved to ../../output/preprocess/Vitamin_D_Levels/gene_data/GSE123993.csv
Loaded clinical data:
                  GSM3518336  GSM3518337  GSM3518338  GSM3518339  GSM3518340  \
Vitamin_D_Levels         1.0         1.0         1.0         1.0         1.0   
Gender                   1.0         1.0         0.0         0.0         0.0   

                  GSM3518341  GSM3518342  GSM3518343  GSM3518344  GSM3518345  \
Vitamin_D_Levels         1.0         1.0         1.0         0.0         0.0   
Gender                   0.0         1.0         1.0         1.0         1.0   

                  ...  GSM3518370  GSM3518371  GSM3518372  GSM3518373  \
Vitamin_D_Levels  ...         1.0         1.0         1.0         1.0   
Gender            ...         1.0         1.0         1.0         1.0   

                  GSM3518374  GSM3518375  GSM3518376  GSM3518377  GSM3518378  \
Vitamin_D_Levels         0.0         0.0         1.0         1.0         0.0   
Gender               

Linked data shape after handling missing values: (44, 23275)
For the feature 'Vitamin_D_Levels', the least common label is '1' with 14 occurrences. This represents 31.82% of the dataset.
The distribution of the feature 'Vitamin_D_Levels' in this dataset is fine.

Is trait biased: False
Data quality check result: Usable


Linked data saved to ../../output/preprocess/Vitamin_D_Levels/GSE123993.csv
